현재 모델의 문제점:
- 우리가 선택한 날씨 독립변수들은 관측값이기에 예보를 하기에는 힘들다.
- 유저들의 입장에서는 예보를 일일이 입력을 하는것 보다는 날짜를 넣었을 때 결과값을 보는것을 원한다

위의 두 문제를 해결하기 위해 예보로 받을 수 있는 데이터를 가지고 다른 전력량 예측 모델을 학습하는것이 목표이다

# 예보 가져오기

In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.request import urlretrieve
from urllib.parse import quote
import os

from dotenv import load_dotenv
load_dotenv()
url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst'
key = os.getenv('serviceKey')
base_date = '20251231'
base_time = '1400'

params ={'serviceKey' : key, 'pageNo' : '1', 'numOfRows' : '1200', 'dataType' : 'XML', 'base_date' : '20251230', 'base_time' : '2000', 'nx' : '60', 'ny' : '127' }

response = requests.get(url, params=params)
soup = BeautifulSoup(response.text, 'xml')

print(len(soup.select('item')))

0


## 예보구분 - 항목 정보

| 항목값 | 항목명 | 단위 | 압축bit수 |
|--------|--------|------|----------|
| POP | 강수확률 | % | 8 |
| PTY | 강수형태 | 코드값 | 4 |
| PCP | 1시간 강수량 | 범주 (1 mm) | 8 |
| REH | 습도 | % | 8 |
| SNO | 1시간 신적설 | 범주(1 cm) | 8 |
| SKY | 하늘상태 | 코드값 | 4 |
| TMP | 1시간 기온 | ℃ | 10 |
| TMN | 일 최저기온 | ℃ | 10 |
| TMX | 일 최고기온 | ℃ | 10 |
| UUU | 풍속(동서성분) | m/s | 12 |
| VVV | 풍속(남북성분) | m/s | 12 |
| WAV | 파고 | M | 8 |
| VEC | 풍향 | deg | 10 |
| WSD | 풍속 | m/s | 10 |

- 기상청 api에서 받을수 있는 예보 정보는 위와 같습니다.
- 또한 4일 동안의 예보를 받아 볼 수 있습니다.

EDA를 통해 우리는 바람(풍속, 풍향)의 데이터는 전력량에 영향을 끼치지 않는 것으로 확인하였으며 기온 중는 최저기온이 강수량 데이터는 습도 데이터가 전력량과 가장 높은 상관관계와 정보량이 있는것을 확인하였습니다. 그렇기에 우리는 최저기온, 습도, 지역코드(구 단위)의 데이터를 학습한 예보 모델을 만들어보겠습니다

In [20]:
# 예보에서 하루 최저기온과 습도 데이터 가져오기
def get_data(date):
    import requests
    from bs4 import BeautifulSoup
    from urllib.request import urlretrieve
    from urllib.parse import quote
    import os
    import pandas as pd
    from datetime import datetime, timedelta
    from dotenv import load_dotenv
    load_dotenv()

    url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst'
    key = os.getenv('serviceKey')

    date = pd.to_datetime(date)
    now = datetime.now()
    if ( now.hour < 3):
        now = now-timedelta(days=1)
    base_date = now.strftime('%Y%m%d')
    hour = now.hour
    if ( now.minute <= 15):
        hour = hour-1
    base_date = now.strftime('%Y%m%d')
    hour = (hour-2)//3*3+2
    if (hour < 0):
        hour = hour+24
    base_time = '{}00'.format(hour)

    params ={'serviceKey' : key, 'pageNo' : '1', 'numOfRows' : '1200', 'dataType' : 'XML', 'base_date' : base_date, 'base_time' : base_time, 'nx' : '60', 'ny' : '127' }

    response = requests.get(url, params=params)
    soup = BeautifulSoup(response.text, 'xml')

    item_list = soup.select('item')
    TMN = 100
    REH = 0
    count = 0
    for item in item_list:
        category = item.select_one('category').text
        if ((category == 'TMP') & (date == pd.to_datetime(item.select_one('fcstDate').text))):
            if TMN > float(item.fcstValue.text):
                TMN = float(item.fcstValue.text)
        if ((category == 'REH') & ((date == pd.to_datetime(item.select_one('fcstDate').text)))):
            if item.fcstValue.text:
                count+=1
                REH += float(item.fcstValue.text)
    
    if count == 0:
        count +=1
    return TMN, REH/count

get_data('2026-01-01')


(100, 0.0)

# 지역(구), 최저기온, 습도로 전력량을 파악하는 모델 만들기

## 예보 모델 훈련 데이터

In [7]:
import numpy as np
import pandas as pd

df = pd.read_csv('../data/훈련데이터셋/1_훈련데이터셋_날짜포함.csv')
df.head()

,지역코드,날짜,최저기온(°C),0.5m 지중온도(°C),평균 증기압(hPa),가조시간(hr),평균 상대습도(%),파워
0,1111010100,2022-12-04,-5.2,7.5,1.8,9.7,38.9,146294.6135
1,1111010100,2022-12-05,-7.0,7.0,2.0,9.7,41.9,175633.8270
2,1111010100,2022-12-06,-3.9,6.6,4.1,9.7,62.8,156084.1910
3,1111010100,2022-12-08,0.1,6.6,4.4,9.7,57.5,177018.8420
4,1111010100,2022-12-09,0.9,6.6,5.1,9.7,60.8,158467.1690


In [8]:
# 실제 사용 데이터만 가져오기
df = df[['지역코드', '날짜', '최저기온(°C)', '평균 상대습도(%)', '파워']]
df.head()

,지역코드,날짜,최저기온(°C),평균 상대습도(%),파워
0,1111010100,2022-12-04,-5.2,38.9,146294.6135
1,1111010100,2022-12-05,-7.0,41.9,175633.8270
2,1111010100,2022-12-06,-3.9,62.8,156084.1910
3,1111010100,2022-12-08,0.1,57.5,177018.8420
4,1111010100,2022-12-09,0.9,60.8,158467.1690


In [9]:
# 휴일여부 추가
df = df.sort_values(['지역코드', '날짜']).reset_index(drop=True)
df['날짜'] = pd.to_datetime(df['날짜'])
df

,지역코드,날짜,최저기온(°C),평균 상대습도(%),파워
0,1111010100,2022-12-04,-5.2,38.9,146294.6135
1,1111010100,2022-12-05,-7.0,41.9,175633.8270
2,1111010100,2022-12-06,-3.9,62.8,156084.1910
3,1111010100,2022-12-08,0.1,57.5,177018.8420
4,1111010100,2022-12-09,0.9,60.8,158467.1690
...,...,...,...,...,...
181403,1174011000,2024-01-24,-11.2,52.0,115124.5470
181404,1174011000,2024-01-25,-9.7,47.8,99907.5000
181405,1174011000,2024-01-26,-6.9,62.5,94818.5120
181406,1174011000,2024-01-27,-4.6,70.0,86792.8280


In [10]:
day_off_list = ['2022-01-01', '2022-01-31', '2022-02-01', '2022-02-02', '2022-03-01', '2022-03-09', '2022-05-05', '2022-05-08', '2022-06-01', '2022-06-06', '2022-08-15', '2022-09-09',
                '2022-09-10', '2022-09-11', '2022-09-12', '2022-10-03', '2022-10-09', '2022-10-10', '2022-12-25', '2023-01-01', '2023-01-21', '2023-01-22', '2023-01-23', '2023-01-24', 
                '2023-03-01', '2023-05-05', '2023-05-27', '2023-06-06', '2023-08-15', '2023-09-28', '2023-09-29', '2023-09-30', '2023-10-03', '2023-10-09', '2023-12-25', '2024-01-01', 
                '2024-02-09', '2024-02-10', '2024-02-11', '2024-02-12', '2024-03-01', '2024-04-10', '2024-05-05', '2024-05-06', '2024-05-15', '2024-06-06', '2024-08-15', '2024-09-16',
                '2024-09-17', '2024-09-18', '2024-10-03', '2024-10-09', '2024-12-25']
day_off_list = pd.to_datetime(day_off_list)
day_off_list

DatetimeIndex(['2022-01-01', '2022-01-31', '2022-02-01', '2022-02-02',
               '2022-03-01', '2022-03-09', '2022-05-05', '2022-05-08',
               '2022-06-01', '2022-06-06', '2022-08-15', '2022-09-09',
               '2022-09-10', '2022-09-11', '2022-09-12', '2022-10-03',
               '2022-10-09', '2022-10-10', '2022-12-25', '2023-01-01',
               '2023-01-21', '2023-01-22', '2023-01-23', '2023-01-24',
               '2023-03-01', '2023-05-05', '2023-05-27', '2023-06-06',
               '2023-08-15', '2023-09-28', '2023-09-29', '2023-09-30',
               '2023-10-03', '2023-10-09', '2023-12-25', '2024-01-01',
               '2024-02-09', '2024-02-10', '2024-02-11', '2024-02-12',
               '2024-03-01', '2024-04-10', '2024-05-05', '2024-05-06',
               '2024-05-15', '2024-06-06', '2024-08-15', '2024-09-16',
               '2024-09-17', '2024-09-18', '2024-10-03', '2024-10-09',
               '2024-12-25'],
              dtype='datetime64[ns]', freq=None

In [13]:
df['휴일여부'] = (df['날짜'].dt.weekday == 5) | (df['날짜'].dt.weekday == 6) | (df['날짜'].isin(day_off_list))
df['휴일여부'] = df['휴일여부'].astype(int)
df.head()

,지역코드,날짜,최저기온(°C),평균 상대습도(%),파워,휴일여부
0,1111010100,2022-12-04,-5.2,38.9,146294.6135,1
1,1111010100,2022-12-05,-7.0,41.9,175633.8270,0
2,1111010100,2022-12-06,-3.9,62.8,156084.1910,0
3,1111010100,2022-12-08,0.1,57.5,177018.8420,0
4,1111010100,2022-12-09,0.9,60.8,158467.1690,0


## 모델 개발을 위해 훈련데이터 나누기

In [14]:
# 결측치 먼저 처리하기
df = df.dropna(axis=0).drop(columns=['날짜']).reset_index(drop=True)
df['지역코드'] = df['지역코드']
df.head()

,지역코드,최저기온(°C),평균 상대습도(%),파워,휴일여부
0,1111010100,-5.2,38.9,146294.6135,1
1,1111010100,-7.0,41.9,175633.8270,0
2,1111010100,-3.9,62.8,156084.1910,0
3,1111010100,0.1,57.5,177018.8420,0
4,1111010100,0.9,60.8,158467.1690,0


In [15]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

X = df.drop(columns='파워')
y = df['파워']

X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size=0.2)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1)
X_train.shape, X_valid.shape, X_test.shape, y_train.shape, y_valid.shape, y_test.shape

((130613, 4), (14513, 4), (36282, 4), (130613,), (14513,), (36282,))

In [16]:
cat_cols = ['지역코드', '휴일여부']
for c in cat_cols:
    X_train[c] = X_train[c].astype("category")
    X_valid[c] = X_valid[c].astype('category')
    X_test[c] = X_test[c].astype("category")

## 모델 생성

In [17]:
from lightgbm import LGBMRegressor
model = LGBMRegressor(
    objective="regression_l2",   # 🔥 반드시 변경

    n_estimators=20000,
    learning_rate=0.03,

    num_leaves=63,
    max_depth=10,
    min_child_samples=80,        # ↓ 완화
    min_split_gain=0.05,         # ↓ 완화

    subsample=0.8,
    subsample_freq=1,
    colsample_bytree=0.8,

    reg_alpha=0.1,               # ↓
    reg_lambda=5.0,              # ↓

    random_state=42,
    n_jobs=-1
)

model.fit(
    X_train, y_train,
    eval_set=[(X_valid, y_valid)],
    eval_metric="rmse",
    categorical_feature=cat_cols if len(cat_cols) > 0 else "auto",
    callbacks=[
        # early stopping (너무 오래 돌지 않게)
        # verbose는 200번마다 한번씩 로그
        __import__("lightgbm").early_stopping(stopping_rounds=200, verbose=True),
        __import__("lightgbm").log_evaluation(period=200),
    ]
)

[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000713 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 954
[LightGBM] [Info] Number of data points in the train set: 130613, number of used features: 4
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 138217.717788
Training until validation scores don't improve for 200 rounds
[200]	valid_0's rmse: 158612	valid_0's l2: 2.51576e+10
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightG

LGBMRegressor(colsample_bytree=0.8, learning_rate=0.03, max_depth=10,
              min_child_samples=80, min_split_gain=0.05, n_estimators=20000,
              n_jobs=-1, num_leaves=63, objective='regression_l2',
              random_state=42, reg_alpha=0.1, reg_lambda=5.0, subsample=0.8,
              subsample_freq=1)

In [18]:
from sklearn.metrics import mean_squared_error, r2_score
def eval_print(name, y_true, y_pred):
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    r2 = r2_score(y_true, y_pred)
    print(f"[{name}] RMSE: {rmse:,.3f}")
    print(f"[{name}] R2  : {r2:.6f}")
    return rmse, r2

pred_valid = model.predict(X_valid)
pred_test  = model.predict(X_test)

eval_print("VALID", y_valid, pred_valid)
eval_print("TEST",  y_test,  pred_test)

print("Best iteration:", model.best_iteration_)

[VALID] RMSE: 157,974.928
[VALID] R2  : 0.746128
[TEST] RMSE: 174,283.325
[TEST] R2  : 0.714114
Best iteration: 321


In [19]:
import joblib
import os
path = '예보LGBM/'
if not os.path.exists(path):
    os.mkdir(path)
joblib.dump(model, path+'final_model.joblib')

['예보LGBM/final_model.joblib']